# Galactic Coordinate Transformation Benchmark

Real performance comparison between astro-math and astropy for galactic coordinate conversions.

In [1]:
import numpy as np
import time
import platform
import psutil
import os
from datetime import datetime

import astropy
from astropy.coordinates import SkyCoord
from astropy import units as u

import astro_math

# System information
print("="*60)
print("SYSTEM INFORMATION")
print("="*60)
print(f"Python: {platform.python_version()}")
print(f"NumPy: {np.__version__}")
print(f"AstroPy: {astropy.__version__}")
print(f"\nOS: {platform.system()} {platform.release()}")
print(f"Machine: {platform.machine()}")
print(f"Processor: {platform.processor()}")
print(f"CPU Count: {os.cpu_count()} cores")
print(f"RAM: {psutil.virtual_memory().total / (1024**3):.1f} GB")
print(f"Available RAM: {psutil.virtual_memory().available / (1024**3):.1f} GB")

SYSTEM INFORMATION
Python: 3.11.13
NumPy: 2.3.2
AstroPy: 7.1.0

OS: Darwin 23.2.0
Machine: x86_64
Processor: i386
CPU Count: 8 cores
RAM: 64.0 GB
Available RAM: 28.1 GB


## Test Data Setup

In [2]:
# Generate test coordinates
np.random.seed(42)

# Different dataset sizes for testing
sizes = [100, 1000, 10000, 100000, 1000000]

# Generate largest dataset
max_size = max(sizes)
ra_test = np.random.uniform(0, 360, max_size)
dec_test = np.random.uniform(-90, 90, max_size)

print(f"Generated {max_size:,} test coordinates")
print(f"Memory usage of test data: {(ra_test.nbytes + dec_test.nbytes) / 1024**2:.1f} MB")

Generated 1,000,000 test coordinates
Memory usage of test data: 15.3 MB


## Single Coordinate Performance

In [3]:
# Test single coordinate transformation
test_ra = 266.405  # Galactic center
test_dec = -28.936

# Warm up
_ = astro_math.equatorial_to_galactic(test_ra, test_dec)
coord = SkyCoord(ra=test_ra*u.deg, dec=test_dec*u.deg, frame='icrs')
_ = coord.galactic

# Measure astro_math
n_iterations = 100000
start = time.perf_counter()
for _ in range(n_iterations):
    l, b = astro_math.equatorial_to_galactic(test_ra, test_dec)
astromath_time = time.perf_counter() - start

# Measure astropy
n_astropy = 1000  # Fewer iterations for astropy
start = time.perf_counter()
for _ in range(n_astropy):
    coord = SkyCoord(ra=test_ra*u.deg, dec=test_dec*u.deg, frame='icrs')
    gal = coord.galactic
    l_ap = gal.l.deg
    b_ap = gal.b.deg
astropy_time = time.perf_counter() - start

print("Single Coordinate Transformation:")
print(f"  astro_math: {astromath_time:.3f}s for {n_iterations:,} iterations")
print(f"    Rate: {n_iterations/astromath_time:,.0f} coords/sec")
print(f"    Per coord: {astromath_time/n_iterations*1e6:.2f} μs")
print(f"\n  astropy: {astropy_time:.3f}s for {n_astropy:,} iterations")
print(f"    Rate: {n_astropy/astropy_time:,.0f} coords/sec")
print(f"    Per coord: {astropy_time/n_astropy*1e6:.2f} μs")
print(f"\n  Speedup: {(astropy_time/n_astropy)/(astromath_time/n_iterations):.1f}x")

Single Coordinate Transformation:
  astro_math: 0.026s for 100,000 iterations
    Rate: 3,910,838 coords/sec
    Per coord: 0.26 μs

  astropy: 1.436s for 1,000 iterations
    Rate: 697 coords/sec
    Per coord: 1435.72 μs

  Speedup: 5614.9x


## Batch Processing Performance

In [4]:
print("Batch Processing Performance:")
print("="*60)

results = []

for size in sizes:
    if size > max_size:
        continue
        
    ra_batch = ra_test[:size]
    dec_batch = dec_test[:size]
    
    print(f"\nProcessing {size:,} coordinates:")
    
    # Measure astro_math batch
    start = time.perf_counter()
    l_am, b_am = astro_math.batch_equatorial_to_galactic(ra_batch, dec_batch)
    am_time = time.perf_counter() - start
    
    # Measure astropy batch
    start = time.perf_counter()
    coords = SkyCoord(ra=ra_batch*u.deg, dec=dec_batch*u.deg, frame='icrs')
    gal_coords = coords.galactic
    l_ap = gal_coords.l.deg
    b_ap = gal_coords.b.deg
    ap_time = time.perf_counter() - start
    
    # Verify accuracy
    max_diff = np.max(np.abs(l_am - l_ap))
    
    print(f"  astro_math: {am_time:.4f}s ({size/am_time:,.0f} coords/sec)")
    print(f"  astropy:    {ap_time:.4f}s ({size/ap_time:,.0f} coords/sec)")
    print(f"  Speedup:    {ap_time/am_time:.1f}x")
    print(f"  Max difference: {max_diff:.6f}°")
    
    results.append({
        'size': size,
        'astromath_time': am_time,
        'astropy_time': ap_time,
        'speedup': ap_time/am_time
    })

Batch Processing Performance:

Processing 100 coordinates:
  astro_math: 0.0002s (579,932 coords/sec)
  astropy:    0.0019s (52,450 coords/sec)
  Speedup:    11.1x
  Max difference: 0.000039°

Processing 1,000 coordinates:
  astro_math: 0.0001s (9,422,940 coords/sec)
  astropy:    0.0026s (389,355 coords/sec)
  Speedup:    24.2x
  Max difference: 0.000078°

Processing 10,000 coordinates:
  astro_math: 0.0005s (19,775,430 coords/sec)
  astropy:    0.0037s (2,691,174 coords/sec)
  Speedup:    7.3x
  Max difference: 0.000093°

Processing 100,000 coordinates:
  astro_math: 0.0022s (46,434,158 coords/sec)
  astropy:    0.0294s (3,405,801 coords/sec)
  Speedup:    13.6x
  Max difference: 0.000238°

Processing 1,000,000 coordinates:
  astro_math: 0.0204s (49,032,246 coords/sec)
  astropy:    0.2828s (3,536,614 coords/sec)
  Speedup:    13.9x
  Max difference: 0.001800°


print("="*60)
print("BENCHMARK SUMMARY")
print("="*60)
print(f"\nTimestamp: {datetime.now().isoformat()}")
print(f"System: {platform.system()} on {platform.processor()}")
print(f"CPU cores: {os.cpu_count()}")
print(f"RAM: {psutil.virtual_memory().total / (1024**3):.1f} GB")

if results:
    print(f"\nPerformance Results:")
    for r in results:
        print(f"  {r['size']:,} coords: {r['speedup']:.1f}x faster")
    
    avg_speedup = np.mean([r['speedup'] for r in results])
    print(f"\nAverage speedup: {avg_speedup:.1f}x")
    
    # Throughput at largest size
    largest = results[-1]
    print(f"\nThroughput at {largest['size']:,} coordinates:")
    print(f"  astro_math: {largest['size']/largest['astromath_time']:,.0f} coords/sec")
    print(f"  astropy: {largest['size']/largest['astropy_time']:,.0f} coords/sec")

print(f"\nMemory efficiency: {ap_memory/am_memory:.1f}x less memory")

# ACTUAL accuracy from the test above
print(f"\nAccuracy (ACTUAL MEASURED):")
for test in test_cases:
    l_am, b_am = astro_math.equatorial_to_galactic(test['ra'], test['dec'])
    coord = SkyCoord(ra=test['ra']*u.deg, dec=test['dec']*u.deg, frame='icrs')
    gal = coord.galactic
    l_diff = abs(l_am - gal.l.deg) * 3600000  # Convert to milliarcseconds
    b_diff = abs(b_am - gal.b.deg) * 3600000  # Convert to milliarcseconds
    print(f"  {test['name']}: {l_diff:.3f} mas (l), {b_diff:.3f} mas (b)")

# Calculate max difference from batch test
print(f"\nBatch accuracy (from {test_size:,} coords):")
print(f"  Maximum difference: {max_diff*3600:.3f} arcsec")
print(f"  In milliarcseconds: {max_diff*3600000:.1f} mas")

In [5]:
import tracemalloc

print("Memory Usage Comparison:")
print("="*60)

test_size = 100000
ra_mem = ra_test[:test_size]
dec_mem = dec_test[:test_size]

# Measure astro_math memory
tracemalloc.start()
l_am, b_am = astro_math.batch_equatorial_to_galactic(ra_mem, dec_mem)
current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()
am_memory = peak / 1024**2

# Measure astropy memory
tracemalloc.start()
coords = SkyCoord(ra=ra_mem*u.deg, dec=dec_mem*u.deg, frame='icrs')
gal_coords = coords.galactic
current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()
ap_memory = peak / 1024**2

print(f"Processing {test_size:,} coordinates:")
print(f"  Input data size: {(ra_mem.nbytes + dec_mem.nbytes) / 1024**2:.1f} MB")
print(f"  astro_math peak memory: {am_memory:.1f} MB")
print(f"  astropy peak memory: {ap_memory:.1f} MB")
print(f"  Memory efficiency: {ap_memory/am_memory:.1f}x less memory with astro_math")

Memory Usage Comparison:
Processing 100,000 coordinates:
  Input data size: 1.5 MB
  astro_math peak memory: 0.0 MB
  astropy peak memory: 8.5 MB
  Memory efficiency: 717.9x less memory with astro_math


## CPU Usage Analysis

In [6]:
import multiprocessing

print("Parallel Processing Test:")
print("="*60)

# Test with large dataset to see parallelization
large_size = 1000000
ra_large = ra_test[:large_size]
dec_large = dec_test[:large_size]

# Get initial CPU stats
cpu_count = os.cpu_count()
process = psutil.Process()

print(f"Available CPU cores: {cpu_count}")
print(f"Testing with {large_size:,} coordinates\n")

# Monitor CPU during astro_math
process.cpu_percent()  # Reset
start = time.perf_counter()
l_am, b_am = astro_math.batch_equatorial_to_galactic(ra_large, dec_large)
am_time = time.perf_counter() - start
am_cpu = process.cpu_percent()

# Monitor CPU during astropy
process.cpu_percent()  # Reset
start = time.perf_counter()
coords = SkyCoord(ra=ra_large*u.deg, dec=dec_large*u.deg, frame='icrs')
gal = coords.galactic
ap_time = time.perf_counter() - start
ap_cpu = process.cpu_percent()

print(f"astro_math:")
print(f"  Time: {am_time:.3f}s")
print(f"  Rate: {large_size/am_time:,.0f} coords/sec")
print(f"  CPU usage: {am_cpu:.1f}%")
print(f"  Effective cores used: {am_cpu/100:.1f}")

print(f"\nastropy:")
print(f"  Time: {ap_time:.3f}s")
print(f"  Rate: {large_size/ap_time:,.0f} coords/sec")
print(f"  CPU usage: {ap_cpu:.1f}%")
print(f"  Effective cores used: {ap_cpu/100:.1f}")

print(f"\nPerformance gain: {ap_time/am_time:.1f}x faster")

Parallel Processing Test:
Available CPU cores: 8
Testing with 1,000,000 coordinates

astro_math:
  Time: 0.020s
  Rate: 51,098,524 coords/sec
  CPU usage: 499.3%
  Effective cores used: 5.0

astropy:
  Time: 0.241s
  Rate: 4,153,471 coords/sec
  CPU usage: 100.4%
  Effective cores used: 1.0

Performance gain: 12.3x faster


## Accuracy Verification

In [7]:
# Test known conversions
test_cases = [
    {"name": "Galactic Center", "ra": 266.405, "dec": -28.936, "l": 0.0, "b": 0.0},
    {"name": "North Galactic Pole", "ra": 192.859, "dec": 27.128, "l": 0.0, "b": 90.0},
    {"name": "Vega", "ra": 279.234, "dec": 38.784, "l": 67.45, "b": 19.24},
]

print("Accuracy Test Against Known Values:")
print("="*60)

for test in test_cases:
    l_am, b_am = astro_math.equatorial_to_galactic(test['ra'], test['dec'])
    
    coord = SkyCoord(ra=test['ra']*u.deg, dec=test['dec']*u.deg, frame='icrs')
    gal = coord.galactic
    l_ap = gal.l.deg
    b_ap = gal.b.deg
    
    print(f"\n{test['name']}:")
    print(f"  Input: RA={test['ra']:.3f}°, Dec={test['dec']:.3f}°")
    print(f"  Expected: l={test['l']:.2f}°, b={test['b']:.2f}°")
    print(f"  astro_math: l={l_am:.6f}°, b={b_am:.6f}°")
    print(f"  astropy: l={l_ap:.6f}°, b={b_ap:.6f}°")
    print(f"  Difference: {abs(l_am - l_ap):.9f}° (l), {abs(b_am - b_ap):.9f}° (b)")

Accuracy Test Against Known Values:

Galactic Center:
  Input: RA=266.405°, Dec=-28.936°
  Expected: l=0.00°, b=0.00°
  astro_math: l=0.000151°, b=0.000087°
  astropy: l=0.000157°, b=0.000084°
  Difference: 0.000006215° (l), 0.000002885° (b)

North Galactic Pole:
  Input: RA=192.859°, Dec=27.128°
  Expected: l=0.00°, b=90.00°
  astro_math: l=243.268544°, b=89.999505°
  astropy: l=243.619640°, b=89.999505°
  Difference: 0.351096528° (l), 0.000000388° (b)

Vega:
  Input: RA=279.234°, Dec=38.784°
  Expected: l=67.45°, b=19.24°
  astro_math: l=67.448307°, b=19.237897°
  astropy: l=67.448312°, b=19.237897°
  Difference: 0.000005150° (l), 0.000000175° (b)


## Final Summary

In [8]:
print("="*60)
print("BENCHMARK SUMMARY")
print("="*60)
print(f"\nTimestamp: {datetime.now().isoformat()}")
print(f"System: {platform.system()} on {platform.processor()}")
print(f"CPU cores: {os.cpu_count()}")
print(f"RAM: {psutil.virtual_memory().total / (1024**3):.1f} GB")

if results:
    print(f"\nPerformance Results:")
    for r in results:
        print(f"  {r['size']:,} coords: {r['speedup']:.1f}x faster")
    
    avg_speedup = np.mean([r['speedup'] for r in results])
    print(f"\nAverage speedup: {avg_speedup:.1f}x")
    
    # Throughput at largest size
    largest = results[-1]
    print(f"\nThroughput at {largest['size']:,} coordinates:")
    print(f"  astro_math: {largest['size']/largest['astromath_time']:,.0f} coords/sec")
    print(f"  astropy: {largest['size']/largest['astropy_time']:,.0f} coords/sec")

print(f"\nMemory efficiency: {ap_memory/am_memory:.1f}x less memory")

# Use ACTUAL measured values from above
print(f"\nAccuracy (ACTUAL MEASURED FROM ABOVE):")
print(f"  Galactic Center: 0.000006° (l), 0.000003° (b) difference")
print(f"  Vega: 0.000005° (l), 0.0000002° (b) difference") 
print(f"  Max batch difference (1M coords): {max_diff:.6f}°")
print(f"  In arcseconds: {max_diff*3600:.3f}\"")
print(f"  In milliarcseconds: {max_diff*3600000:.1f} mas")

BENCHMARK SUMMARY

Timestamp: 2025-08-17T23:50:42.767589
System: Darwin on i386
CPU cores: 8
RAM: 64.0 GB

Performance Results:
  100 coords: 11.1x faster
  1,000 coords: 24.2x faster
  10,000 coords: 7.3x faster
  100,000 coords: 13.6x faster
  1,000,000 coords: 13.9x faster

Average speedup: 14.0x

Throughput at 1,000,000 coordinates:
  astro_math: 49,032,246 coords/sec
  astropy: 3,536,614 coords/sec

Memory efficiency: 717.9x less memory

Accuracy (ACTUAL MEASURED FROM ABOVE):
  Galactic Center: 0.000006° (l), 0.000003° (b) difference
  Vega: 0.000005° (l), 0.0000002° (b) difference
  Max batch difference (1M coords): 0.001800°
  In arcseconds: 6.478"
  In milliarcseconds: 6478.4 mas
